In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import os

import nltk


In [61]:
## Pull data

df_full = pd.read_pickle('../data/raw_data1.pkl')
df_presconf = df_full[df_full['variable'] == 'fomc_presconf']

df_presconf.shape

(69, 6)

In [62]:
replacement_dict = {'BECKNE R' : 'BECKNER',
'MIC HELLE' : 'MICHELLE',
'MICHEL LE' : 'MICHELLE',
'POWE LL' : 'POWELL',
'PRELIM INARY' : 'PRELIMINARY',
'E CI' : 'ECI',
'P CE' : 'PCE',
'MICH ELLE' : 'MICHELLE',
'N AIRU' : 'NAIRU',
'CHA IR' : 'CHAIR',
'HOWAR D' : 'HOWARD',
'PAUL LA' : 'PAULLA',
'MICHE LLE' : 'MICHELLE',
'MICHELL E' : 'MICHELLE',
'SMI ALEK' : 'SMIALEK',
'S CHNEIDER' : 'SCHNEIDER',
'CH EUNG' : 'CHEUNG',
'GENZ ER' : 'GENZER',
'SMI TH' : 'SMITH',
'CRR A' : 'CRRA',
'CHRIS TOPHER' : 'CHRISTOPHER',
'CHRIST OPHER' : 'CHRISTOPHER',
'PO WELL' : 'POWELL',
'SM ITH' : 'SMITH',
'R RP' : 'RRP',
'C ARES' : 'CARES',
'SMI TH' : 'SMITH',
'P OWELL' : 'POWELL',
'SCOT T' : 'SCOTT',
'CH EUNG' : 'CHEUNG',
'M BS' : 'MBS',
'MB S' : 'MBS',
'L ONG' : 'LONG',
'USMC A' : 'USMCA',
'IM F':'IMF',
'POW ELL' : 'POWELL',
'MARS HALL' : 'MARSHALL',
'POWEL L' : 'POWELL',
'P OWELL' : 'POWELL',
'CHAI R' : 'CHAIR',
'CHA IR' : 'CHAIR',
'PUZZANGHER A' : 'PUZZANGHERA',
'CHAI R' : 'CHAIR',
'G DP' : 'GDP',
'C NBC' : 'CNBC',
'AP PELBAUM' : 'APPELBAUM',
'YELLE N' : 'YELLEN',
'GILL ESPIE' : 'GILLESPIE',
'SE P' : 'SEP',
'S EP' : 'SEP',
'NAR A' : 'NARA',
'FDI C' : 'FDIC',
'CF PB' : 'CFPB',
'P BS' : 'PBS',
'MART IN' : 'MARTIN',
'CHA IR' : 'CHAIR',
'LIESMA N' : 'LIESMAN',
'APP ELBAUM' : 'APPELBAUM',
'YE LLEN' : 'YELLEN',
'SOM A' : 'SOMA',
'LIBO R' : 'LIBOR',
'FINA L' : 'FINAL',
'CHAIR MAN' : 'CHAIRMAN',
'CHAIRM AN' : 'CHAIRMAN',
'BERN ANKE' : 'BERNANKE',
'BERNA NKE' : 'BERNANKE',
'BER NANKE' : 'BERNANKE',
'APPE LBAUM' : 'APPELBAUM',
'S EC' : 'SEC',
'GRE G' : 'GREG',
'ANDREW S' : 'ANDREWS',
'GREG I P' : 'GREG IP',
'F OMC' : 'FOMC',
'FO MC' : 'FOMC',
'FOM C' : 'FOMC',
'PUZZA NGHERA' : 'PUZZANGHERA',
'KE ARNS' : 'KEARNS',
'HILSENRAT H' : 'HILSENRATH',
'FI CO' : 'FICO',
'PC E' : 'PCE',
'C OSTA' : 'COSTA',
'IMF I': 'IMFI',
'NAN CY' : 'NANCY',
'QUESTION' : 'UNIDENTIFIED SPEAKER'}

In [63]:
## Step 0: Correct broken words
df_presconf['text'] = df_presconf['text'].replace(replacement_dict, regex=True)


C:\Users\Prayut Jain\AppData\Local\Temp\ipykernel_14412\3140125650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_presconf['text'] = df_presconf['text'].replace(replacement_dict, regex=True)


In [64]:
df_presconf.head()

,date,source,text,title,chairman,variable
593,2011-04-27,fed_reserve,"April 27, 2011 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf
594,2011-06-22,fed_reserve,"June 22, 2011 Chairman Bernanke’s Press Confer...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf
595,2011-11-02,fed_reserve,"November 2, 2011 Chairman Bernanke’s Press Co...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf
596,2012-01-25,fed_reserve,"January 25, 2012 Chairman Bernanke’s Press Co...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf
597,2012-04-25,fed_reserve,"April 25, 2012 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf


In [65]:
def extract_speakers(text):
    pattern = r'\b([A-Z]+\s[A-Z]+(?:\s[A-Z]+)?)\b'
    pats = list(set(re.findall(pattern, text)))
    pat_ret = [pat for pat in pats if pat not in ['MICHELLE SMITH']]
    return pat_ret

def breakdown_transcript(row):
    text = row['text']
    chairman = row['chairman'].split('.')[0].upper()
    speakers = extract_speakers(text)
    speakers.sort(key=text.find)

    # If the only speakers are UNIDENTIFIED SPEAKER and CHAIRMAN
    if set(speakers) == {'UNIDENTIFIED SPEAKER', 'CHAIRMAN BERNANKE'}:
        segments = [text.split('UNIDENTIFIED SPEAKER')[0].strip()]  # Chairman's opening remarks
        qas = text.split('UNIDENTIFIED SPEAKER')[1:]  # Questions and answers
        for qa in qas:
            segments.append('UNIDENTIFIED SPEAKER' + qa.split(chairman, 1)[0].strip())  # Question
            if chairman in qa:
                segments[-1] += " " + chairman + qa.split(chairman, 1)[1].strip()  # Answer
        return segments

    # If only the chairman is present or no speakers found
    if len(speakers) <= 1:
        return [text.strip()]

    # Handle opening remarks
    opening_remark, text = text.split(speakers[1], 1)
    segments = [opening_remark.strip()]

    for i in range(1, len(speakers) - 1):
        segment, text = text.split(speakers[i + 1], 1)
        segments.append(speakers[i] + segment)
    segments.append(speakers[-1] + text)

    # Combine segments if chairman answers a question
    combined_segments = []
    i = 0
    while i < len(segments):
        if chairman in segments[i] and i != 0:  # Combine with the previous segment if chairman answers
            combined_segments[-1] += " " + segments[i]
        else:
            combined_segments.append(segments[i])
        i += 1

    return combined_segments

# Apply the function to the dataframe
df_presconf['segments'] = df_presconf.apply(breakdown_transcript, axis=1)
expanded_df = df_presconf.explode('segments').reset_index(drop=True)

# Create a 'segment' column
expanded_df['segment'] = expanded_df.groupby('date').cumcount()
expanded_df['segment'] = expanded_df['segment'].apply(lambda x: 'opening_remark' if x == 0 else f'qa_pair{x}')

expanded_df.columns = ['date', 'source', 'full_text', 'title', 'chairman', 'variable', 'segment_text', 'segment']

print(expanded_df)

           date       source  \
0    2011-04-27  fed_reserve   
1    2011-04-27  fed_reserve   
2    2011-04-27  fed_reserve   
3    2011-04-27  fed_reserve   
4    2011-04-27  fed_reserve   
...         ...          ...   
1271 2023-05-03  fed_reserve   
1272 2023-05-03  fed_reserve   
1273 2023-05-03  fed_reserve   
1274 2023-05-03  fed_reserve   
1275 2023-06-14  fed_reserve   

                                              full_text  \
0     April 27, 2011 Chairman Bernanke’s Press Confe...   
1     April 27, 2011 Chairman Bernanke’s Press Confe...   
2     April 27, 2011 Chairman Bernanke’s Press Confe...   
3     April 27, 2011 Chairman Bernanke’s Press Confe...   
4     April 27, 2011 Chairman Bernanke’s Press Confe...   
...                                                 ...   
1271  May 3, 2023   Chair Powell ’s Press Conference...   
1272  May 3, 2023   Chair Powell ’s Press Conference...   
1273  May 3, 2023   Chair Powell ’s Press Conference...   
1274  May 3, 2023   Chair

C:\Users\Prayut Jain\AppData\Local\Temp\ipykernel_14412\3792825960.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_presconf['segments'] = df_presconf.apply(breakdown_transcript, axis=1)


In [66]:
expanded_df['date'].value_counts()

2018-09-26    25
2013-12-18    24
2019-01-30    24
2012-06-20    24
2018-06-13    23
              ..
2022-11-02    14
2020-03-15    14
2014-09-17    13
2020-03-03     9
2023-06-14     1
Name: date, Length: 69, dtype: int64

In [67]:
expanded_df.head()

,date,source,full_text,title,chairman,variable,segment_text,segment
0,2011-04-27,fed_reserve,"April 27, 2011 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf,"April 27, 2011 Chairman Bernanke’s Press Confe...",opening_remark
1,2011-04-27,fed_reserve,"April 27, 2011 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf,"UNIDENTIFIED SPEAKER. Mr. Chairman, tomorrow ...",qa_pair1
2,2011-04-27,fed_reserve,"April 27, 2011 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf,"UNIDENTIFIED SPEAKER. Mr. Chairman, given wha...",qa_pair2
3,2011-04-27,fed_reserve,"April 27, 2011 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf,"UNIDENTIFIED SPEAKER. Mr. Chairman, first, th...",qa_pair3
4,2011-04-27,fed_reserve,"April 27, 2011 Chairman Bernanke’s Press Confe...",Press Conference Transcript,Ben S. Bernanke,fomc_presconf,"UNIDENTIFIED SPEAKER. I’m sorry, Mr. Chairman...",qa_pair4


In [68]:
expanded_df.to_csv('../data/df_presconf.csv')